<a href="https://colab.research.google.com/github/lizhjort/Insight/blob/master/goodreads_shape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import lightgbm as lgb

In [0]:
data_with_checkouts = pd.read_pickle('/content/gdrive/My Drive/insight_data/data_10_col.pkl.gz')

In [0]:
goodreads = pd.read_csv('/content/gdrive/My Drive/insight_data/good_reads.csv')

In [0]:
goodreads.head()

,id,isbn,isbn13,ratings_count,reviews_count,text_reviews_count,work_ratings_count,work_reviews_count,work_text_reviews_count,average_rating,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18
0,30841956,316273880,9.780320e+12,1196,5332,208,16652,40881,1837,4.22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,31248728,1101870052,9.781100e+12,226,1362,33,270,1501,37,4.04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,30841952,316262870,9.780320e+12,333,1251,77,414,1443,88,3.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,34078056,1410499642,9.781410e+12,398,20,70,387780,607254,40345,4.32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,32075854,62645226,9.780060e+12,33479,47061,5321,46901,55282,6761,3.93,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
data_with_checkouts.head()

,BibNumber,CheckoutDate,CheckoutCount,CheckoutCount1WeekPrev,CheckoutCount2WeekPrev,CheckoutCountNextWeek,Genre,Title,ISBN,PublicationYear
0,10143,2017-09-03,0.0,NaN,NaN,0.0,72.0,The price / by Arthur Miller..,"082220911X, 9780822209119",c1997.
1,10143,2017-09-10,0.0,0.0,NaN,0.0,72.0,The price / by Arthur Miller..,"082220911X, 9780822209119",c1997.
2,10143,2017-09-17,0.0,0.0,0.0,0.0,72.0,The price / by Arthur Miller..,"082220911X, 9780822209119",c1997.
3,10143,2017-09-24,0.0,0.0,0.0,0.0,72.0,The price / by Arthur Miller..,"082220911X, 9780822209119",c1997.
4,10143,2017-10-01,0.0,0.0,0.0,0.0,72.0,The price / by Arthur Miller..,"082220911X, 9780822209119",c1997.


In [0]:
split_isbn = data_with_checkouts['ISBN'].str.split(",", n=1, expand=True)

In [0]:
split_isbn.drop([1], axis=1, inplace=True)

In [0]:
data_with_checkouts['isbn'] = split_isbn

In [0]:
data_with_checkouts.head()

,BibNumber,CheckoutDate,CheckoutCount,CheckoutCount1WeekPrev,CheckoutCount2WeekPrev,CheckoutCountNextWeek,Genre,Title,ISBN,PublicationYear,isbn
0,10143,2017-09-03,0.0,NaN,NaN,0.0,72.0,The price / by Arthur Miller..,"082220911X, 9780822209119",c1997.,082220911X
1,10143,2017-09-10,0.0,0.0,NaN,0.0,72.0,The price / by Arthur Miller..,"082220911X, 9780822209119",c1997.,082220911X
2,10143,2017-09-17,0.0,0.0,0.0,0.0,72.0,The price / by Arthur Miller..,"082220911X, 9780822209119",c1997.,082220911X
3,10143,2017-09-24,0.0,0.0,0.0,0.0,72.0,The price / by Arthur Miller..,"082220911X, 9780822209119",c1997.,082220911X
4,10143,2017-10-01,0.0,0.0,0.0,0.0,72.0,The price / by Arthur Miller..,"082220911X, 9780822209119",c1997.,082220911X


In [0]:
goodreads.drop(['work_ratings_count', 'work_reviews_count', 'work_text_reviews_count', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18'], axis=1)

,id,isbn,isbn13,ratings_count,reviews_count,text_reviews_count,average_rating
0,30841956,316273880,9.780320e+12,1196,5332,208,4.22
1,31248728,1101870052,9.781100e+12,226,1362,33,4.04
2,30841952,316262870,9.780320e+12,333,1251,77,3.75
3,34078056,1410499642,9.781410e+12,398,20,70,4.32
4,32075854,62645226,9.780060e+12,33479,47061,5321,3.93
5,29940815,674545494,9.780670e+12,12,191,2,4.00
6,32025434,691172579,9.780690e+12,0,10,0,0.00
7,30215481,1504038754,9.781500e+12,120,749,49,3.83
8,33276897,1943370109,9.781940e+12,32,87,5,4.37
9,30653593,62491989,9.780060e+12,346,1050,58,4.15


In [0]:
goodreads.dtypes

id                           int64
isbn                        object
isbn13                     float64
ratings_count                int64
reviews_count                int64
text_reviews_count           int64
work_ratings_count           int64
work_reviews_count           int64
work_text_reviews_count      int64
average_rating             float64
Unnamed: 10                float64
Unnamed: 11                float64
Unnamed: 12                float64
Unnamed: 13                float64
Unnamed: 14                float64
Unnamed: 15                float64
Unnamed: 16                float64
Unnamed: 17                float64
Unnamed: 18                float64
dtype: object

In [0]:
goodreads.shape

(93887, 19)

In [0]:
goodreads = goodreads.drop_duplicates()

In [0]:
goodreads = goodreads.drop(['id', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18'], axis=1)


In [0]:
goodreads

,isbn,isbn13,ratings_count,reviews_count,text_reviews_count,work_ratings_count,work_reviews_count,work_text_reviews_count,average_rating
0,316273880,9.780320e+12,1196,5332,208,16652,40881,1837,4.22
1,1101870052,9.781100e+12,226,1362,33,270,1501,37,4.04
2,316262870,9.780320e+12,333,1251,77,414,1443,88,3.75
3,1410499642,9.781410e+12,398,20,70,387780,607254,40345,4.32
4,62645226,9.780060e+12,33479,47061,5321,46901,55282,6761,3.93
5,674545494,9.780670e+12,12,191,2,12,191,2,4.00
6,691172579,9.780690e+12,0,10,0,0,10,0,0.00
7,1504038754,9.781500e+12,120,749,49,190,931,55,3.83
8,1943370109,9.781940e+12,32,87,5,1210,3355,140,4.37
9,62491989,9.780060e+12,346,1050,58,1043,2450,99,4.15


In [0]:
goodreads.to_pickle('/content/gdrive/My Drive/insight_data/goodreads.pkl.gz')

In [0]:
data_with_checkouts.to_pickle('/content/gdrive/My Drive/insight_data/data_10_col.pkl.gz')